# Santander Bank Recommender System Demo
## Data Preparation for AWS Personalize

This notebook is the first step in creating a demo bank recommender system using AWS Personalize with the Kaggle Santander dataset. We'll prepare the three required datasets for AWS Personalize:

1. Users Dataset
2. Items Dataset
3. Interactions Dataset

These datasets will be derived from the raw Santander dataset file downloaded from kaggle and stored in S3: 
https://www.kaggle.com/competitions/santander-product-recommendation/code

## Setup and Data Loading

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
from io import StringIO
import boto3
import json
import time
import os

from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Set pandas display options for better output readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [2]:
# Initialize a session using Amazon S3
s3 = boto3.client('s3')

In [3]:
# Set up your S3 bucket and file name
bucket_name = 'souhail-work-bucket'
file_key = 'personalize_santander_raw_data/train_ver2.csv'  # Change this to your actual file path in the S3 bucket
local_file_path = 'train_ver2.csv'   # Local file path where we want to save the file


# Download the file from S3 to the local file system
with open(local_file_path, 'wb') as f:
    s3.download_fileobj(bucket_name, file_key, f)

# Load the data into a pandas DataFrame
df = pd.read_csv(local_file_path)

/tmp/ipykernel_591/2996586977.py:12: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(local_file_path)


In [4]:
# Check the size of the dataset
df.size

655070832

The dataset is quite large. For demo purposes and to reduce processing time, we'll sample a subset of the data.

In [5]:
# Sample the dataset for faster processing
# Note: In a production environment, you might want to use the full dataset
df = df.sample(n=1000000, random_state=42)

In [6]:
# Display the first few rows and basic information about the dataset
print(df.head())
print(df.info())

# This gives us an overview of the data types and non-null counts for each column.
# We'll use this information to decide which features to include in our datasets.

          fecha_dato  ncodpers ind_empleado pais_residencia sexo  age  \
2406613   2015-04-28    886976            N              ES    H   49   
6466267   2015-10-28    950206            N              ES    H   24   
10239289  2016-02-28    474137            N              ES    V   58   
6681075   2015-10-28   1236387            N              ES    V   34   
2305701   2015-04-28   1359789            N              ES    H   22   

          fecha_alta  ind_nuevo antiguedad  indrel ult_fec_cli_1t indrel_1mes  \
2406613   2010-07-27        0.0         60     1.0            NaN         1.0   
6466267   2011-09-28        0.0         49     1.0            NaN         1.0   
10239289  2004-05-14        0.0        141     1.0            NaN         1.0   
6681075   2014-01-07        0.0         21     1.0            NaN         1.0   
2305701   2014-11-24        0.0          8     1.0            NaN         1.0   

         tiprel_1mes indresi indext conyuemp canal_entrada indfall  tipodo

## Create Users Dataset

The Users dataset will contain information about each customer. We'll select relevant features that could influence product recommendations.

In [7]:
# Prepare User dataset with additional features
def prepare_user_dataset(df):
    user_df = df[['ncodpers', 'age', 'antiguedad', 'renta', 'ind_nuevo', 'segmento', 
                  'ind_empleado', 'pais_residencia', 'sexo', 'fecha_alta', 
                  'ind_actividad_cliente', 'nomprov']].drop_duplicates()
    
    user_df.columns = ['USER_ID', 'AGE', 'CUSTOMER_TENURE', 'INCOME', 'NEW_CUSTOMER', 
                       'SEGMENT', 'EMPLOYEE_INDEX', 'COUNTRY_RESIDENCE', 'SEX', 
                       'FIRST_HOLD_DATE', 'CUSTOMER_ACTIVITY_INDEX', 'PROVINCE_NAME']
    
    user_df['USER_ID'] = user_df['USER_ID'].astype(str)
    
    # Convert 'FIRST_HOLD_DATE' to datetime and calculate account age
    user_df['FIRST_HOLD_DATE'] = pd.to_datetime(user_df['FIRST_HOLD_DATE'])
    user_df['ACCOUNT_AGE_DAYS'] = (pd.Timestamp('2015-06-28') - user_df['FIRST_HOLD_DATE']).dt.days
    
    # Drop the original 'FIRST_HOLD_DATE' column
    user_df = user_df.drop('FIRST_HOLD_DATE', axis=1)
    
    return user_df

user_dataset = prepare_user_dataset(df)
print(user_dataset.head())
print(user_dataset.info())


          USER_ID  AGE CUSTOMER_TENURE     INCOME  NEW_CUSTOMER  \
2406613    886976   49              60   98343.03           0.0   
6466267    950206   24              49        NaN           0.0   
10239289   474137   58             141        NaN           0.0   
6681075   1236387   34              21        NaN           0.0   
2305701   1359789   22               8  126999.66           0.0   

                     SEGMENT EMPLOYEE_INDEX COUNTRY_RESIDENCE SEX  \
2406613    02 - PARTICULARES              N                ES   H   
6466267   03 - UNIVERSITARIO              N                ES   H   
10239289   02 - PARTICULARES              N                ES   V   
6681075    02 - PARTICULARES              N                ES   V   
2305701   03 - UNIVERSITARIO              N                ES   H   

          CUSTOMER_ACTIVITY_INDEX PROVINCE_NAME  ACCOUNT_AGE_DAYS  
2406613                       1.0        BURGOS            1797.0  
6466267                       0.0       BADAJO

### Explanation of User Dataset Columns

- USER_ID: Unique identifier for each user
- AGE: User's age
- CUSTOMER_TENURE: How long the customer has been with the bank
- INCOME: Customer's income
- NEW_CUSTOMER: Indicates if the customer is new
- SEGMENT: Customer segmentation category
- EMPLOYEE_INDEX: Indicates if the customer is an employee
- COUNTRY_RESIDENCE: Customer's country of residence
- SEX: Customer's gender
- CUSTOMER_ACTIVITY_INDEX: Index of customer activity
- PROVINCE_NAME: Name of the province
- ACCOUNT_AGE_DAYS: Number of days since the account was opened

These features provide a comprehensive profile of each customer, which AWS Personalize can use to generate more accurate recommendations.

## Create Items Dataset

The Items dataset will contain information about each product offered by the bank. In this case, our items are the various financial products indicated by the 'ind_*_fin_ult1' columns in the original dataset.

In [8]:
# Prepare Items dataset
def prepare_items_dataset(df):
    product_columns = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
                       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
                       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
                       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
                       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
                       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
                       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
                       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']
    
    items_df = pd.DataFrame({'ITEM_ID': product_columns})
    
    # Dictionary mapping product codes to explicit English descriptions
    product_descriptions = {
        'ind_ahor_fin_ult1': 'Savings',
        'ind_aval_fin_ult1': 'Guarantees',
        'ind_cco_fin_ult1': 'CurrentAccount',
        'ind_cder_fin_ult1': 'Derivatives',
        'ind_cno_fin_ult1': 'PayrollAccount',
        'ind_ctju_fin_ult1': 'JuniorAccount',
        'ind_ctma_fin_ult1': 'MasAccount',
        'ind_ctop_fin_ult1': 'ParticularAccount',
        'ind_ctpp_fin_ult1': 'ParticularPlusAccount',
        'ind_deco_fin_ult1': 'ShortDeposit',
        'ind_deme_fin_ult1': 'MediumDeposit',
        'ind_dela_fin_ult1': 'LongDeposit',
        'ind_ecue_fin_ult1': 'eAccount',
        'ind_fond_fin_ult1': 'Funds',
        'ind_hip_fin_ult1': 'Mortgage',
        'ind_plan_fin_ult1': 'Pensions',
        'ind_pres_fin_ult1': 'Loans',
        'ind_reca_fin_ult1': 'Taxes',
        'ind_tjcr_fin_ult1': 'CreditCard',
        'ind_valo_fin_ult1': 'Securities',
        'ind_viv_fin_ult1': 'HomeAccount',
        'ind_nomina_ult1': 'Payroll',
        'ind_nom_pens_ult1': 'PensionPayments',
        'ind_recibo_ult1': 'DirectDebit'
    }
    
    items_df['PRODUCT_DESCRIPTION'] = items_df['ITEM_ID'].map(product_descriptions)
    
    return items_df

items_dataset = prepare_items_dataset(df)
print(items_dataset)

              ITEM_ID    PRODUCT_DESCRIPTION
0   ind_ahor_fin_ult1                Savings
1   ind_aval_fin_ult1             Guarantees
2    ind_cco_fin_ult1         CurrentAccount
3   ind_cder_fin_ult1            Derivatives
4    ind_cno_fin_ult1         PayrollAccount
5   ind_ctju_fin_ult1          JuniorAccount
6   ind_ctma_fin_ult1             MasAccount
7   ind_ctop_fin_ult1      ParticularAccount
8   ind_ctpp_fin_ult1  ParticularPlusAccount
9   ind_deco_fin_ult1           ShortDeposit
10  ind_deme_fin_ult1          MediumDeposit
11  ind_dela_fin_ult1            LongDeposit
12  ind_ecue_fin_ult1               eAccount
13  ind_fond_fin_ult1                  Funds
14   ind_hip_fin_ult1               Mortgage
15  ind_plan_fin_ult1               Pensions
16  ind_pres_fin_ult1                  Loans
17  ind_reca_fin_ult1                  Taxes
18  ind_tjcr_fin_ult1             CreditCard
19  ind_valo_fin_ult1             Securities
20   ind_viv_fin_ult1            HomeAccount
21    ind_

### Explanation of Items Dataset

The Items dataset contains two columns:
- ITEM_ID: Unique identifier for each product (corresponds to the original column names)
- PRODUCT_DESCRIPTION: A more readable description of the product

This dataset provides a clear mapping between the product codes used in the interactions and their actual descriptions, which will be useful for interpreting the recommendations generated by AWS Personalize.

## Create Interactions Dataset

The Interactions dataset will contain information about which products each user has. In this dataset, we'll consider a product as 'interacted with' if the corresponding column in the original dataset has a value of 1.

In [9]:
# Prepare Interactions dataset
def prepare_interactions_dataset(df, items_df):
    interactions = []
    
    for _, row in df.iterrows():
        user_id = str(row['ncodpers'])
        timestamp = int(datetime.strptime(row['fecha_dato'], '%Y-%m-%d').timestamp())
        
        for item_id in items_df['ITEM_ID']:
            if row[item_id] == 1:
                interactions.append({
                    'USER_ID': user_id,
                    'ITEM_ID': item_id,
                    'TIMESTAMP': timestamp,
                    'EVENT_TYPE': 'purchase'
                })
    
    interactions_df = pd.DataFrame(interactions)
    return interactions_df

interactions_dataset = prepare_interactions_dataset(df, items_dataset)
print(interactions_dataset.head())
print(interactions_dataset.info())

  USER_ID            ITEM_ID   TIMESTAMP EVENT_TYPE
0  886976   ind_cno_fin_ult1  1430179200   purchase
1  886976  ind_ctma_fin_ult1  1430179200   purchase
2  886976  ind_ecue_fin_ult1  1430179200   purchase
3  886976  ind_tjcr_fin_ult1  1430179200   purchase
4  950206   ind_cco_fin_ult1  1445990400   purchase
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1461842 entries, 0 to 1461841
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   USER_ID     1461842 non-null  object
 1   ITEM_ID     1461842 non-null  object
 2   TIMESTAMP   1461842 non-null  int64 
 3   EVENT_TYPE  1461842 non-null  object
dtypes: int64(1), object(3)
memory usage: 44.6+ MB
None


### Explanation of Interactions Dataset

The Interactions dataset contains four columns:
- USER_ID: Identifies the user involved in the interaction
- ITEM_ID: Identifies the product involved in the interaction
- TIMESTAMP: The time when the interaction occurred (converted to Unix timestamp)
- EVENT_TYPE: Type of interaction (in this case, always 'purchase')

This dataset captures the relationship between users and products. Each row represents a user having a specific product (indicated by a '1' in the original dataset), which we interpret as a 'purchase' event.

Note: In this simplified model, we're treating all product ownerships as 'purchases' at the time recorded in the dataset. In a more sophisticated model, you might want to differentiate between different types of interactions or track changes over time.

In [11]:
# Export datasets to CSV locally
user_dataset.to_csv('users.csv', index=False)
items_dataset.to_csv('items.csv', index=False)
interactions_dataset.to_csv('interactions.csv', index=False)

print("Datasets exported to CSV files.")

Datasets exported to CSV files.


In [14]:
# Export datasets to CSV on S3
bucket_name = 'souhail-work-bucket'

def dataframe_to_s3(df, bucket, key):
    """
    Save a pandas DataFrame to S3 as a CSV file
    """
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    s3.put_object(Bucket=bucket, Key=key, Body=csv_buffer.getvalue())
    print(f"Saved {key} to S3 bucket {bucket}")


# Save datasets to S3
dataframe_to_s3(user_dataset, bucket_name, 'personalize-data/Users.csv')
dataframe_to_s3(items_dataset, bucket_name, 'personalize-data/Items.csv')
dataframe_to_s3(interactions_dataset, bucket_name, 'personalize-data/Interactions.csv')

print("All datasets have been successfully saved to S3.")

# Generate S3 URLs for the datasets
s3_base_url = f"s3://{bucket_name}/personalize-data/"
print(f"\nS3 URLs for your datasets:")
print(f"Users dataset: {s3_base_url}Users.csv")
print(f"Items dataset: {s3_base_url}Items.csv")
print(f"Interactions dataset: {s3_base_url}Interactions.csv")

# Optional: List contents of the S3 bucket to verify
response = s3.list_objects_v2(Bucket=bucket_name, Prefix='personalize-data/')
print("\nContents of S3 bucket:")
for obj in response.get('Contents', []):
    print(f"- {obj['Key']}")

Saved personalize-data/users.csv to S3 bucket souhail-work-bucket
Saved personalize-data/items.csv to S3 bucket souhail-work-bucket
Saved personalize-data/interactions.csv to S3 bucket souhail-work-bucket
All datasets have been successfully saved to S3.

S3 URLs for your datasets:
Users dataset: s3://souhail-work-bucket/personalize-data/Users.csv
Items dataset: s3://souhail-work-bucket/personalize-data/Items.csv
Interactions dataset: s3://souhail-work-bucket/personalize-data/Interactions.csv

Contents of S3 bucket:
- personalize-data/Interactions.csv
- personalize-data/Items.csv
- personalize-data/Users.csv
- personalize-data/interactions.csv
- personalize-data/items.csv
- personalize-data/users.csv


## Conclusion

We have successfully prepared three datasets for use with AWS Personalize:

1. Users Dataset: Contains information about each customer
2. Items Dataset: Contains information about each product offered by the bank
3. Interactions Dataset: Contains information about which products each user has

These datasets are now ready to be uploaded to Amazon S3 and used to create a recommendation model in AWS Personalize.

Next steps:
1. Create a dataset group in AWS Personalize
2. Import the datasets into AWS Personalize
3. Create and train a recommendation model
4. Generate and evaluate recommendations

Remember to monitor your AWS usage and costs when working with large datasets and training models.